In [1]:
import pulp
import pandas as pd

In [2]:
T = ['Arsenal',
     'AstonVilla',
     'Bournemouth',
     'Brentford',
     'Brighton',
     'Chelsea',
     'CrystalPalace',
     'Everton',
     'Fulham',
     'LeedsUnited',
     'LeicesterCity',
     'Liverpool',
     'ManchesterCity',
     'ManchesterUnited',
     'NewcastleUnited',
     'NottinghamForest',
     'Southampton',
     'TottenhamHotspur',
     'WestHamUnited',
     'WolverhamptonWanderers'
     ]

In [3]:
W = range(1, 2*len(T) - 1)
W2 = range(1, 2*len(T) - 3)
W_first_half = range(1, len(T))
W_second_half = range(len(T), 2*len(T) - 1)

In [4]:
# Decision variable indicating team h plays against team a in week w
x = pulp.LpVariable.dicts('x', (W, T, T), cat=pulp.LpBinary)

In [5]:
# Team cannot play itself
for w in W:

    for h in T:

        for a in T:

            if h == a:

                del x[w][h][a]

schedule = pulp.LpProblem("Schedule", pulp.LpMinimize)

In [6]:
# Not yet decided on objective function
schedule += 0

In [7]:
# Each team plays exactly one match each week
for w in W:

    for i in T:

        schedule += pulp.lpSum(x[w][i][a] for a in T if a != i) + \
            pulp.lpSum(x[w][h][i] for h in T if h != i) == 1

In [8]:
# Teams play each other twice during the season, with one home match and one away match
for i in T:
    for j in T:
        if j != i:
            # team i plays one match in the first half against team j
            schedule += pulp.lpSum(x[w][i][j] for w in W_first_half) + \
                pulp.lpSum(x[w][j][i] for w in W_first_half) == 1
            # team i plays one match in the second half against team j
            schedule += pulp.lpSum(x[w][i][j] for w in W_second_half) + \
                pulp.lpSum(x[w][j][i] for w in W_second_half) == 1
            # if team i played at home against team j in the first half, they will play away against team j in the second half, and vice versa
            schedule += pulp.lpSum(x[w][i][j] for w in W_first_half) == pulp.lpSum(
                x[w][j][i] for w in W_second_half)
            schedule += pulp.lpSum(x[w][j][i] for w in W_first_half) == pulp.lpSum(
                x[w][i][j] for w in W_second_half)

In [9]:
# First two and last two games cannot be consecutive away or home matches
for i in T:
    schedule += pulp.lpSum(x[W[0]][i][j] for j in T if i != j) + pulp.lpSum(x[W[1]][i][j] for j in T if i != j) == 1
    schedule += pulp.lpSum(x[W[-1]][i][j] for j in T if i != j) + pulp.lpSum(x[W[-2]][i][j] for j in T if i != j) == 1

In [10]:
# If a team is home during boxing day (week 17) it will be away during new year's day (week 18)
for i in T:
    schedule += pulp.lpSum(x[W[16]][i][j] for j in T if i != j) + pulp.lpSum(x[W[17]][i][j] for j in T if i != j) == 1

In [11]:
schedule.solve()

1

In [12]:
# initialize a dictionary to store the match locations for each team
team_dict = {i: {week: '-' for week in W} for i, team in enumerate(T)}

for w in W:

    for h in T:

        for a in T:

            if a != h:

                if x[w][h][a].value() == 1.0:
                    week, home, away = x[w][h][a].getName().split('_')[1:]
                    team_dict[T.index(home)][int(
                        week)] = f'{T.index(away) + 1}/H'
                    team_dict[T.index(away)][int(
                        week)] = f'{T.index(home) + 1}/A'
                    print(f'Week: {week}  {home} vs. {away}')

    print()

Week: 1  AstonVilla vs. NewcastleUnited
Week: 1  Chelsea vs. TottenhamHotspur
Week: 1  Everton vs. WestHamUnited
Week: 1  LeicesterCity vs. LeedsUnited
Week: 1  Liverpool vs. Brighton
Week: 1  ManchesterCity vs. Fulham
Week: 1  ManchesterUnited vs. Arsenal
Week: 1  NottinghamForest vs. Bournemouth
Week: 1  Southampton vs. CrystalPalace
Week: 1  WolverhamptonWanderers vs. Brentford

Week: 2  Arsenal vs. NottinghamForest
Week: 2  Bournemouth vs. AstonVilla
Week: 2  Brentford vs. Chelsea
Week: 2  Brighton vs. WolverhamptonWanderers
Week: 2  CrystalPalace vs. ManchesterCity
Week: 2  Fulham vs. ManchesterUnited
Week: 2  LeedsUnited vs. Everton
Week: 2  NewcastleUnited vs. LeicesterCity
Week: 2  TottenhamHotspur vs. Southampton
Week: 2  WestHamUnited vs. Liverpool

Week: 3  AstonVilla vs. Chelsea
Week: 3  Brentford vs. Fulham
Week: 3  Everton vs. Brighton
Week: 3  LeedsUnited vs. WestHamUnited
Week: 3  Liverpool vs. LeicesterCity
Week: 3  ManchesterCity vs. Arsenal
Week: 3  ManchesterUnited 

In [13]:
# create the dataframe from the dictionary
df = pd.DataFrame(team_dict)
# transpose the dataframe
df_t = df.transpose()

df_t.to_csv('PremierLeagueSchedule2022-2023.csv', index=False)